In [59]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
print(os.listdir("../input"))

['test.csv', 'train.csv', 'sample_submission.csv']


In [60]:
train = pd.read_csv('../input/train.csv')#, nrows=10000)
test = pd.read_csv('../input/test.csv')

test.head(10)



,id,comment_text
0,7000000,Jeff Sessions is another one of Trump's Orwell...
1,7000001,I actually inspected the infrastructure on Gra...
2,7000002,No it won't . That's just wishful thinking on ...
3,7000003,Instead of wringing our hands and nibbling the...
4,7000004,how many of you commenters have garbage piled ...
5,7000005,Why can't the Globe & Mail provide the symbols...
6,7000006,"That's already been happening, Carl, it's call..."
7,7000007,"imagine the costs for security, transportation..."
8,7000008,So they want to put United on the board of thi...
9,7000009,"No, no that can't be. It's Russia that's the ..."


In [61]:
# adding preprocessing from this kernel: https://www.kaggle.com/taindow/simple-cudnngru-python-keras
punct_mapping = {"_":" ", "`":" "}
punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'
def clean_special_chars(text, punct, mapping):
    for p in mapping:
        text = text.replace(p, mapping[p])    
    for p in punct:
        text = text.replace(p, f' {p} ')     
    return text
train['comment_text'] = train['comment_text'].apply(lambda x: clean_special_chars(x, punct, punct_mapping))
test['comment_text'] = test['comment_text'].apply(lambda x: clean_special_chars(x, punct, punct_mapping))

In [62]:
identity_columns = ['male', 'female', 'homosexual_gay_or_lesbian', 'christian', 'jewish', 'muslim', 'black', 'white', 'psychiatric_or_mental_illness']
for col in identity_columns + ['target']:
    train[col] = np.where(train[col] >= 0.5, True, False)

In [63]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(train['comment_text'], 
                                                    train['target'], 
                                                    random_state=0)

In [64]:
from sklearn.feature_extraction.text import TfidfVectorizer


vect = TfidfVectorizer(min_df=3,ngram_range=(1, 2), max_features=30000).fit(X_train)


X_train_vectorized = vect.transform(X_train)


In [68]:
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import LogisticRegression


model = KernelRidge()#alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)

#model = LogisticRegression()

model.fit(X_train_vectorized, y_train)

predictions = model.predict(vect.transform(X_test))



In [69]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error
#print (mean_squared_error(y_test, predictions))
print (roc_auc_score(y_test, predictions))

0.834463141025641


In [67]:
submission = pd.read_csv('../input/sample_submission.csv', index_col='id')
submission['prediction'] = model.predict(vect.transform(test['comment_text']))
submission.to_csv('submission.csv')